In [1]:
from h2ogpte import H2OGPTE

In [2]:
f = open('tmp/h2ogpte_key_hatespeechdetect.txt', 'r')
API_KEY = f.read()
f.close()

In [3]:
client = H2OGPTE(
    address="https://playground.h2ogpte.h2o.ai",
    api_key=API_KEY)
client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.0-dev4`.
You can enable strict version checking by passing strict_version_check=True.


In [20]:
models = client.get_llms()
len(models)

16

In [22]:
[m['base_model'] for m in models]

['h2oai/h2ogpt-4096-llama2-70b-chat',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.2',
 'h2oai/h2o-danube2-1.8b-chat',
 'SeaLLMs/SeaLLM-7B-v2.5',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'gemini-1.5-pro-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview']

In [4]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'de38d561-554a-43bc-8a16-28a59bd1f0c5'

In [12]:
#MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
MODEL = "gpt-4-1106-preview"
SYS_PROMPT = "あなたは日本語で会話を実施するAIです。返答は必ず日本語です。"

In [31]:
with client.connect(chat_session_id) as session:
    print(session)
    print(type(session))

<class 'h2ogpte.session.Session'>


In [28]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message = 'こんにちは、私の名前は山田です。覚えておいて下さい。',
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)
    
    print("-----------------")
    
    answer = session.query(
        message = '私の名前はなんでした？',
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)

こんにちは、山田さん。もちろん、お名前は覚えておきます。どういったご用件でしょうか？
-----------------
私はあなたの名前を知ることができません。あなたが以前にお名前を教えていない限り、私にはわかりません。もし教えていただければ、今後の会話でお名前を使うことができます。


In [29]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message = 'こんにちは、私の名前は山田です。覚えておいて下さい。',
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)
    
print("-----------------")

with client.connect(chat_session_id) as session2:
    answer = session2.query(
        message = '私の名前はなんでした？',
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)

こんにちは、山田さん。もちろん、お名前は覚えておきます。どういったご用件でしょうか？
-----------------
私はあなたの名前を知ることができません。あなたが以前にお名前を教えていない限り、私にはわかりません。もし教えていただければ、その名前でお呼びすることができます。


In [27]:
answer = client.answer_question(
    llm=MODEL,
    question='こんにちは、私の名前は山田です。',
    system_prompt=SYS_PROMPT,
)
answer

Answer(content='こんにちは、山田さん。お会いできて嬉しいです。何かお手伝いできることはありますか？', error='', prompt_raw='こんにちは、私の名前は山田です。', llm='gpt-4-1106-preview', input_tokens=46, output_tokens=37, origin='answer_question_using_context')

In [26]:
answer.content

"Konnichiwa, Yamada-san! Ogenki desu ka? (こんにちは、山田さん！お元気ですか？)\n\n* Konnichiwa (こんにちは) - Hello\n* Ogenki desu ka (お元気ですか) - How are you?\n\nI'm glad to meet you, Yamada-san! I'm doing well, thanks for asking. How about you?"